# Scheduler = service to start pipeline at specified times

# Orchestrator = System to ensure tasks in a data pipeline are run in the correct order 

# Airflow is both a scheduler and an orchestrator

# Define data pipeline as a DAG

In [35]:
%%capture
! docker compose down

In [36]:
! docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [37]:
# do this in a terminal in this directory
# sudo mkdir -p logs plugins temp dags tests data visualization && sudo chmod -R u=rwx,g=rwx,o=rwx logs plugins temp dags tests data visualization

In [38]:
%%capture
! docker compose up --build -d

In [39]:
! sleep 30

In [44]:
! rm -rf ./dags/tpch_warehouse/models/*/.ipynb_checkpoints # always run before dbt run, caused by notebooks, no need to do this if performed via terminal

In [41]:
! docker ps

CONTAINER ID   IMAGE                                           COMMAND                  CREATED              STATUS                        PORTS                                       NAMES
fc03ce69ee87   6-scheduling--orchestration-airflow-scheduler   "/usr/bin/dumb-init …"   About a minute ago   Up 57 seconds (healthy)       8080/tcp                                    scheduler
a3dfe1c2abc5   6-scheduling--orchestration-airflow-webserver   "/usr/bin/dumb-init …"   About a minute ago   Up 42 seconds (healthy)       0.0.0.0:8080->8080/tcp, :::8080->8080/tcp   webserver
15f6641953e4   postgres:16                                     "docker-entrypoint.s…"   About a minute ago   Up About a minute (healthy)   0.0.0.0:5432->5432/tcp, :::5432->5432/tcp   postgres


In [ ]:
%%capture
! docker compose down